In [ ]:
# Celda 1
# Importaciones estándar
import os
import time
import logging
from datetime import datetime

# Librerías de terceros
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

# Importaciones internas del proyecto
from src.utils import configurar_logs, validar_numero, cargar_excel, normalizar_nombres_columnas, generar_mensaje_alerta
from src.whatsapp import NotificarMensajeWhatsApp

# Configurar logs solo si este archivo es el principal
if __name__ == "__main__":
    configurar_logs(log_file="whatsapp_message_sender.log", log_to_console=True)

# Ruta base del proyecto
base_path = os.getcwd()

# Nombre del compañero Cami
# ---------------------------------------------------
cami_nombre = "Cristian"
# ---------------------------------------------------

# Crear una instancia de la clase para enviar mensajes por WhatsApp
# whatsapp = NotificarMensajeWhatsApp()

In [ ]:
# Celda 2
# Nombres de los archivos originales
archivos_excel = {
    "directorio": os.path.join(base_path, "directorio.xlsx"),
    "data": os.path.join(base_path, "data.xlsx")
}

# Cargar los archivos Excel
archivos_excel = {nombre: cargar_excel(ruta) for nombre, ruta in archivos_excel.items()}

# Acceder a los DataFrames directamente
df_directorio = archivos_excel["directorio"]
df_data_estaciones = archivos_excel["data"]

In [ ]:
# Celda 3
# Normalizar nombres de columnas en todos los DataFrames
df1 = normalizar_nombres_columnas(df_directorio)
df2 = normalizar_nombres_columnas(df_data_estaciones)

# Verificar resultado
print("Columnas normalizadas de df1:", df1.columns)
print("Columnas normalizadas de df2:", df2.columns)

In [ ]:
# Celda 4
# Unir los dos DataFrames en un solo tablero usando 'Estacion' como clave
df_unido = pd.merge(df1, df2, on="id_pc", how="inner")

# Mostrar las primeras filas del DataFrame unido
print("DataFrame unido:")
display(df_unido)


In [ ]:
# Celda 5
# Reemplazar valores vacíos con NaN
df_unido["promedio_de_cloro__mg_l_"] = df_unido["promedio_de_cloro__mg_l_"].replace("", pd.NA)

# Convertir a float
df_unido["promedio_de_cloro__mg_l_"] = pd.to_numeric(df_unido["promedio_de_cloro__mg_l_"], errors="coerce")

# Redondear los valores a 4 decimales
df_unido["promedio_de_cloro__mg_l_"] = df_unido["promedio_de_cloro__mg_l_"].round(4)

# Definir condiciones para la columna 'estado'
condiciones = [
    df_unido["promedio_de_cloro__mg_l_"].isna(),  # Si es NaN → "inactivo"
    df_unido["promedio_de_cloro__mg_l_"] < 0.5,  # Si es menor a 0.5 → "inadecuado"
    df_unido["promedio_de_cloro__mg_l_"] >= 0.5  # Si es mayor o igual a 0.5 → "adecuado"
]

# Mostrar solo el primer nombre del representante
df_unido["representante"] = df_unido["representante"].str.split().str[0]

# Definir valores
valores = np.array(["inactivo", "inadecuado", "adecuado"], dtype=object)
df_unido["estado"] = np.select(condiciones, valores, default="desconocido")

# Columna "Macro" en orden alfabetico (A-Z)
df_unido = df_unido.sort_values(by="macro", ascending=True)

# Verificación
display(df_unido)


In [ ]:
# Celda 6
# Obtener la hora actual en formato HHMM (ej. 0920 o 1000)
hora_actual = datetime.now().replace(second=0, microsecond=0).time()
# hora_actual = datetime.strptime('09:00', '%H:%M').time()
hora_formateada = hora_actual.strftime('%H%M')

print("Hora_actual (HH:MM):", hora_actual)

# # Definir la carpeta de salida
# output_dir = os.path.join(base_path, "data_procesada")
# os.makedirs(output_dir, exist_ok=True)  # Crear carpeta si no existe

# # Nombre del archivo con formato HHMM
# nombre_archivo = f"data_procesada_{hora_formateada}.xlsx"
# ruta_archivo = os.path.join(output_dir, nombre_archivo)

# # Guardar el DataFrame en Excel
# df_unido.to_excel(ruta_archivo, index=False)

# print(f"Archivo guardado en: {ruta_archivo}")


FILTROS

In [ ]:
# Celda 7
# Filtro 1
# ---------------------------------------------------
# cami_nombre = "Cristian"
# ---------------------------------------------------
df_filtrado = df_unido[df_unido["macro"] == cami_nombre]

# Mostrar resultado
display(df_filtrado)  # Solo en Jupyter Notebook

In [ ]:
# Celda 8
# Filtro 2
df_filtrado = df_filtrado[df_filtrado["estado"] != "adecuado"]
df_filtrado = df_filtrado[df_filtrado["estado"] != "inactivo"]

# Mostrar resultado
display(df_filtrado)  # Solo en Jupyter Notebook

In [ ]:
# Celda 9
# Filtro 3

# Convertir 'hora_inferior' y 'hora_superior' a datetime.time
df_filtrado["hora_inferior_dt"] = pd.to_datetime(df_filtrado["hora_inferior"], format="%H:%M").dt.time
df_filtrado["hora_superior_dt"] = pd.to_datetime(df_filtrado["hora_superior"], format="%H:%M").dt.time


# Filtrar filas donde 'hora_inferior' <= 'hora_actual' <= 'hora_superior'
df_filtrado = df_filtrado[hora_actual >= df_filtrado["hora_superior_dt"]]

# Eliminar columnas temporales después del filtrado
df_filtrado = df_filtrado.drop(columns=["hora_inferior_dt", "hora_superior_dt"])

# Mostrar resultado
display(df_filtrado)  # Para Jupyter Notebook


In [ ]:
# df_filtrado.to_excel(f"data_procesada/data_filtrada_{hora_formateada}.xlsx", index=False)

PREPARANDO GATILLADOR

In [ ]:
# Celda 10 - Validar números de teléfono

# Reemplazar el valor de 'telefono' por uno de prueba
# df_filtrado["telefono"] = "+51925122591"  # Asegurar formato correcto

# Aplicar la función de validación en la columna 'telefono'
df_filtrado["telefono_valido"] = df_filtrado["telefono"].apply(validar_numero)

# Contar números válidos e inválidos
num_validos = df_filtrado["telefono_valido"].sum()
num_invalidos = len(df_filtrado) - num_validos

print(f"📞 Números válidos: {num_validos}")
print(f"❌ Números inválidos: {num_invalidos}")

In [ ]:
# Celda 11 - Filtrar solo teléfonos válidos

# Filtrar solo las filas con teléfonos válidos
df_validos = df_filtrado[df_filtrado["telefono_valido"]].copy()

# Extraer listas de teléfonos
telefonos_validos = df_validos["telefono"].tolist()

print(f"✅ Se han filtrado y validado {len(df_validos)} teléfonos del directorio.")

In [ ]:
# Celda 12 - Generar mensajes de alerta

# Lista para almacenar los mensajes generados
mensajes_validos = []

# Iterar sobre el DataFrame filtrado y generar mensajes solo para teléfonos válidos
for idx, row in df_filtrado[df_filtrado["telefono_valido"]].iterrows():
    mensaje = generar_mensaje_alerta(
        telefono=row["telefono"],
        representante_estacion=row["representante"],
        cami_nombre=row["macro"],
        cami_sede=row["sede"],
        estado=row["estado"],
        estacion=row["estacion"],
        ultima_fecha=row.get("ultima_fecha", ""),
        hora_inferior=row["hora_inferior"],
        hora_superior=row["hora_superior"],
        nivel_cloro=row["promedio_de_cloro__mg_l_"],
        localidad=row.get("localidad", ""),
        departamento=row.get("departamento", "")
    )

    # Agregar solo si el mensaje no es None
    if mensaje:
        mensajes_validos.append((row["telefono"], mensaje))

# Mostrar la cantidad de mensajes generados
print(f"📩 Total de mensajes generados: {len(mensajes_validos)}")

# Imprimir los primeros mensajes generados para verificación
for i, (tel, texto) in enumerate(mensajes_validos):
    print(f"\n📌 Mensaje {i+1} para {tel}:")
    print(texto)
    print("------")

    # # Limitar impresión a los primeros 3 mensajes si hay muchos
    # if i >= 2 and len(mensajes_validos) > 3:
    #     print(f"... y {len(mensajes_validos) - 3} mensajes más.")
    #     break


In [ ]:
# Celda 13 - Enviar mensajes por WhatsApp Web
# Extraer listas de teléfonos
telefonos_validos = df_validos["telefono"].tolist()
# Inicializar el enviador de mensajes
sender = NotificarMensajeWhatsApp()

# 1. Inicializar la sesión de WhatsApp Web (solo una vez)
if sender.iniciar_sesion():
    print("✅ Sesión de WhatsApp Web inicializada correctamente.")
else:
    print("❌ Error al inicializar la sesión de WhatsApp Web.")

# 2. Enviar mensajes en lote
try:
    # Enviar todos los mensajes válidos en lote
    sender.enviar_mensajes_en_lote(telefonos_validos, [msg for _, msg in mensajes_validos])

    # Mostrar números que fallaron si los hay
    if sender.numeros_fallidos:
        print("\n🚨 Números que fallaron:")
        for tel, error in sender.numeros_fallidos:
            print(f"❌ {tel}: {error}")
finally:
    # Cerrar sesión y liberar recursos
    sender.cerrar()
